# Gradient descent - MPOs calcualte cohomology
Created 12/05/2024

Objectives:
* Load previous solutions and check.
* Calculate fermionic cohomology.
* Then calculate proj rep cohomology.

# Package imports

In [1]:
import sys

In [2]:
sys.path.append("../")

In [59]:
from functools import reduce
from operator import mul

from collections import namedtuple

import time
import pickle

In [4]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [5]:
import numpy as np
import pandas as pd

rng = np.random.default_rng()
import matplotlib.pyplot as plt

In [6]:
import re

In [7]:
from SPTOptimization.utils import (
    get_right_identity_environment_from_tp_tensor,
    get_left_identity_environment_from_tp_tensor,
    to_npc_array,
    get_physical_dim
)

from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

# Load data

In [8]:
DATA_DIR_1 = r"../data/interpolated_trivial_to_nontrivial_fermionic_trivial_proj_rep_200_site_dmrg/"
DATA_DIR_2 = r"../data/interpolated_nontrivial_fermionic_proj_rep_to_nontrivial_proj_rep_200_site_dmrg/"

In [9]:
def parse_file_name(file_name):
    interpolation = int(file_name.split('_')[0])/100

    return interpolation

In [10]:
loaded_data_triv_proj_rep = dict()
energies_triv_proj_rep = dict()

for local_file_name in list(os.walk(DATA_DIR_1))[0][2]:
    f_name = r"{}/{}".format(DATA_DIR_1, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data_triv_proj_rep[data_info]=data['wavefunction']
        energies_triv_proj_rep[data_info]=data['energy']

In [11]:
loaded_data_non_triv_proj_rep = dict()
energies_non_triv_proj_rep = dict()

for local_file_name in list(os.walk(DATA_DIR_2))[0][2]:
    f_name = r"{}/{}".format(DATA_DIR_2, local_file_name, ignore_unknown=False)

    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data_non_triv_proj_rep[data_info]=data['wavefunction']
        energies_non_triv_proj_rep[data_info]=data['energy']

# Definitons

In [12]:
MAX_VIRTUAL_BOND_DIM = 30
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

Define bosonic symmetries. Label by the group element added.

In [13]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [14]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [15]:
bosonic_npc_symmetries = [
    to_npc_array(X) for X in bosonic_np_symmetries
]

In [16]:
npc_00 = to_npc_array(np_00)
npc_01 = to_npc_array(np_01)
npc_10 = to_npc_array(np_10)
npc_11 = to_npc_array(np_11)

Define "fermionic symmetries". Just identity and JW string.

In [17]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [18]:
fermionic_np_symmetries = [np_I, np_JW]

In [19]:
fermionic_npc_symmetries = [
    to_npc_array(X) for X in fermionic_np_symmetries
]

In [20]:
npc_JW = fermionic_npc_symmetries[1]

In [21]:
symmetry_actions = [
    [[b, f] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [22]:
shifted_symmetry_actions = [
    [[f, b] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [23]:
cases_triv_proj_rep = dict()

for k, psi in loaded_data_triv_proj_rep.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*40,
                left_symmetry_index=60,
                left_boundary_unitaries=[np_I, np_00]*1,
                right_boundary_unitaries=[np_00, np_I]*1
            )

            cases_triv_proj_rep[(k, i, j)] = case

In [24]:
for c in cases_triv_proj_rep.values():
    c.compute_svd_approximate_expectation()

In [25]:
cases_non_triv_proj_rep = dict()

for k, psi in loaded_data_non_triv_proj_rep.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                s*40,
                left_symmetry_index=60,
                left_boundary_unitaries=[np_I, np_00]*1,
                right_boundary_unitaries=[np_00, np_I]*1
            )

            cases_non_triv_proj_rep[(k, i, j)] = case

In [26]:
for c in cases_non_triv_proj_rep.values():
    c.compute_svd_approximate_expectation()

In [27]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [28]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

## Functions

In [29]:
def get_physical_dim(tensor, p_label='p'):
    index = tensor.get_leg_index(p_label)
    dim = tensor.shape[index]
    return dim

In [30]:
def mpo_frobenius_inner_product(mpo1_tensors, mpo2_tensors=None):
    if mpo2_tensors is None:
        mpo2_tensors = mpo1_tensors

    w1 = mpo1_tensors[0]
    dim = get_physical_dim(w1, p_label='p')
    w2 = mpo2_tensors[0]

    t = npc.tensordot(w1, w2.conj(), [['p', 'p*'], ['p*', 'p']])
    #t /= dim

    for w1, w2 in zip(mpo1_tensors[1:], mpo2_tensors[1:]):
        dim = get_physical_dim(w1, p_label='p')

        t = npc.tensordot(t, w1, [['vR', ], ['vL']])
        t = npc.tensordot(t, w2.conj(), [['vR*', 'p', 'p*'], ['vL*', 'p*', 'p']])
        #t /= dim

    return t

In [31]:
def unitary_order_two_right_tensors(w_tensors):
    out = list()

    w = w_tensors[-1]
    t = npc.tensordot(w, w.conj(), [['p', 'p*'], ['p*', 'p']])

    out.append(t)

    for w in w_tensors[-2:0:-1]:
        t = npc.tensordot(t, w, [['vL',], ['vR']])
        t = npc.tensordot(t, w.conj(), [['vL*', 'p', 'p*'], ['vR*', 'p*', 'p']])

        out.append(t)

    return out[::-1]

In [32]:
def unitary_order_four_right_tensors(w_tensors):
    out = list()

    w = w_tensors[-1]
    t = npc.tensordot(w, w.conj(), [['p',], ['p*',]])
    t.ireplace_labels(['vL', 'vL*'], ['vL1', 'vL1*'])
    t = npc.tensordot(t, w, [['p',], ['p*',]])
    t = npc.tensordot(t, w.conj(), [['p', 'p*'], ['p*', 'p']])
    
    out.append(t)

    for w in w_tensors[-2:0:-1]:
        t = npc.tensordot(t, w, [['vL',], ['vR',]])
        t = npc.tensordot(t, w.conj(), [['vL*', 'p'], ['vR*', 'p*']])

        w = w.replace_label('vL', 'vL1')
    
        t = npc.tensordot(t, w, [['vL1', 'p',], ['vR', 'p*']])
        t = npc.tensordot(t, w.conj(), [['vL1*', 'p', 'p*'], ['vR*', 'p*', 'p']])

        out.append(t)

    return out[::-1]

In [33]:
def overlap_right_tensors(w_tensors, b_tensors):
    out = list()

    t = get_right_identity_environment_from_tp_tensor(b_tensors[-1])

    out.append(t)

    # First site
    b = b_tensors[-1]
    w = w_tensors[-1]
    
    t = npc.tensordot(t, b, [['vL',], ['vR',]])
    t = npc.tensordot(
        t,
        w.replace_label('vL', 'vLm'),
        [['p',], ['p*',]]
    )
    t = npc.tensordot(t, b.conj(), [['p', 'vL*',], ['p*', 'vR*',]])

    out.append(t)

    # Inner sites
    for w, b in zip(w_tensors[-2:0:-1], b_tensors[-2:0:-1]):
        t = npc.tensordot(t, b, [['vL',], ['vR',]])
        t = npc.tensordot(
            t,
            w.replace_label('vL', 'vLm'),
            [['p', 'vLm'], ['p*', 'vR']]
        )
        t = npc.tensordot(t, b.conj(), [['p', 'vL*',], ['p*', 'vR*',]])
    
        out.append(t)

    # Last site
    b = b_tensors[0]
    w = w_tensors[0]

    t = npc.tensordot(t, b, [['vL',], ['vR',]])
    t = npc.tensordot(
        t,
        w,
        [['p', 'vLm'], ['p*', 'vR']]
    )
    t = npc.tensordot(t, b.conj(), [['p', 'vL*',], ['p*', 'vR*',]])

    out.append(t)

    return out[::-1]

### Initialize tensors

In [34]:
def rescale_mpo_tensors(mpo_tensors, new_norm):
    num_sites = len(mpo_tensors)

    old_norm = mpo_frobenius_inner_product(mpo_tensors).real
    
    scale_factor = np.power(
        new_norm/old_norm,
        1/(2*num_sites)
    )

    for i in range(num_sites):
        mpo_tensors[i] = scale_factor*mpo_tensors[i]

In [35]:
def generate_random_w_tensor(physical_dim, left_virtual_dim=None,
                             right_virtual_dim=None):

    if (left_virtual_dim is None) and (right_virtual_dim is None):
        dims = (physical_dim, physical_dim)
    elif (left_virtual_dim is None):
        dims = (physical_dim, physical_dim, right_virtual_dim)
    elif (right_virtual_dim is None):
        dims = (physical_dim, physical_dim, left_virtual_dim)
    else: 
        dims = (
            physical_dim,
            physical_dim,
            left_virtual_dim,
            right_virtual_dim
        )
    
    X1 = rng.normal(size=dims)
    X2 = 1j*rng.normal(size=dims)
    X = X1 + X2

        
    if (left_virtual_dim is None) and (right_virtual_dim is None):
        out = npc.Array.from_ndarray_trivial(X, labels=['p', 'p*'])
    elif right_virtual_dim is None:
        out = npc.Array.from_ndarray_trivial(X, labels=['p', 'p*', 'vL'])
    elif left_virtual_dim is None:
        out = npc.Array.from_ndarray_trivial(X, labels=['p', 'p*', 'vR'])
    else:
        out = npc.Array.from_ndarray_trivial(
            X,
            labels=['p', 'p*', 'vL', 'vR']
        )

    return out

In [36]:
def get_random_mpo_tensors(num_sites, norm=None):
    """
    Needs more arguments
    """

    w = generate_random_w_tensor(edge_dims)
    w_tensor = npc.Array.from_ndarray_trivial(
        w,
        labels=['p', 'p*', 'vR']
    )

    w_tensors = [w_tensor,]

    for _ in range(num_sites-2):
        w = generate_random_w_tensor(inner_dims)
        w_tensor = npc.Array.from_ndarray_trivial(w, labels=['p', 'p*', 'vL', 'vR'])
        
        w_tensors.append(w_tensor)

    w = generate_random_w_tensor(edge_dims)
    w_tensor = npc.Array.from_ndarray_trivial(
        w,
        labels=['p', 'p*', 'vL']
    )
    w_tensors.append(w_tensor)

    if norm is not None:
        rescale_mpo_tensors(w_tensors, norm)

    return w_tensors

In [37]:
def get_identity_w_tensor(physical_dim, left_virtual_dim=None, right_virtual_dim=None):
    diagonal = np.ones(physical_dim, dtype='complex')
    identity_matrix = np.diag(diagonal)
    
    if (left_virtual_dim is None) and (right_virtual_dim is None):
        w_tensor = npc.Array.from_ndarray_trivial(
            identity_matrix,
            labels=['p', 'p*']
        )
    elif right_virtual_dim is None:
        X = np.zeros(
            (physical_dim, physical_dim, left_virtual_dim),
            dtype='complex'
        )
        X[...,0] = identity_matrix
        w_tensor = npc.Array.from_ndarray_trivial(
            X,
            labels=['p', 'p*', 'vL']
        )
    elif left_virtual_dim is None:
        X = np.zeros(
            (physical_dim, physical_dim, right_virtual_dim),
            dtype='complex'
        )
        X[...,0] = identity_matrix
        w_tensor = npc.Array.from_ndarray_trivial(
            X,
            labels=['p', 'p*', 'vR']
        )
    else:
        X = np.zeros(
            (physical_dim, physical_dim, left_virtual_dim, right_virtual_dim),
            dtype='complex'
        )
        X[...,0,0] = identity_matrix
        w_tensor = npc.Array.from_ndarray_trivial(
            X,
            labels=['p', 'p*', 'vL', 'vR']
        )

    return w_tensor

In [38]:
def get_random_mpo_tensors(physical_dims, virtual_dims):
    """
    Could fold in with get_identity_mpo_tensors
    """

    w_tensors = [
        generate_random_w_tensor(p_dim, *v_dims)
        for p_dim, v_dims in zip(physical_dims, virtual_dims)
    ]

    return w_tensors

In [39]:
def get_identity_mpo_tensors(physical_dims, virtual_dims):
    """
    Needs more comments, docs!

    Often physical dims, virtual dims will be the same, so could add
    optional behaviour...
    """

    w_tensors = [
        get_identity_w_tensor(p_dim, *v_dims)
        for p_dim, v_dims in zip(physical_dims, virtual_dims)
    ]

    return w_tensors

### ADAM functions

In [40]:
def squared_components(X):
    r, i = (X.real, X.imag)
    return r**2 + 1j*(i**2)

In [41]:
def square_root_components(X):
    r, i = (X.real, X.imag)
    return np.sqrt(r) + 1j*np.sqrt(i)

In [42]:
class AdamTenpy:
    def __init__(self, alpha=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8):
        self.alpha = alpha
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon

        self.moment_1 = None
        self.moment_2 = None

    def update(self, grad):
        if self.moment_1 is None:
            self.moment_1 = (1-self.beta_1)*grad
        else:
            self.moment_1 = self.beta_1*self.moment_1 + (1-self.beta_1)*grad

        grad_squared = grad.unary_blockwise(squared_components)
        if self.moment_2 is None:
            self.moment_2 = (1-self.beta_2)*grad_squared
        else:
            self.moment_2 = self.beta_2*self.moment_2 + (1-self.beta_2)*grad_squared

        self.moment_1 /= (1-self.beta_1)
        self.moment_2 /= (1-self.beta_2)

        out_grad_denom = (
            self.moment_2
            .unary_blockwise(square_root_components)
            .unary_blockwise(lambda x: x + (1+1j)*self.epsilon)
        )

        out_grad = (
            self.moment_1
            .binary_blockwise(np.divide, out_grad_denom)
        )

        return self.alpha*out_grad

### Sweep function

In [43]:
def mpo_tensor_raw_to_gradient(raw_mpo_tensor, gradient_target_tensor):
    """
    Update raw_mpo_tensor calculated to be the gradient by changing leg names
    as neeeded so can be easily and consistently added to
    gradient_target_tensor.
    """
    leg_labels = raw_mpo_tensor.get_leg_labels()

    # First update the virtual legs
    old_new_leg_label_pairs = [
        ('vL*', 'vR'),
        ('vR*', 'vL'),
        ('vL1*', 'vR'),
        ('vR1*', 'vL'),
        ('vLm', 'vR'),
        ('vRm', 'vL'),
    ]

    for old, new in old_new_leg_label_pairs:
        if old in leg_labels:
            raw_mpo_tensor.ireplace_label(old, new)

    # Then create new array to get the physical legs correct.
    # Is this consistent? Should set order of leg labels on raw_mpo_tensor
    # before casting to array?
    out = npc.Array.from_ndarray_trivial(
        raw_mpo_tensor.to_ndarray(),
        labels=raw_mpo_tensor.get_leg_labels()
    )

    out.itranspose(gradient_target_tensor.get_leg_labels())

    return out

In [44]:
def update_mpo_score(raw_gradient_mpo_tensor, gradient_target_tensor,
                     virtual_legs=[['vL*', 'vR*',], ['vR*', 'vL*',]],
                     take_abs=False
                    ):
    # Calculate "score" (just the contraction of the two relevant tensors)
    raw_legs, target_legs = virtual_legs

    score = npc.tensordot(
        raw_gradient_mpo_tensor,
        gradient_target_tensor.conj(),
        [['p', 'p*', *raw_legs], ['p*', 'p', *target_legs]]
    )

    if take_abs:
        real_score = np.abs(score)
    else:
        real_score = score.real

    return score

In [45]:
def mpo_gradient_descent_sweep(mpo_tensors, b_tensors, total_dimension,
    right_overlap_tensors, unitarity_learning_rate, overlap_learning_rate,
    overlap_target, left_environment, adam_optimizers):
    """
    Really need to tidy this function up...

    Also get matrix_dim directly from tensors.
    """
    # Initialise list of gradients to be filled
    grads = list()

    # Initialise variables
    right_unitary_two_tensors = unitary_order_two_right_tensors(mpo_tensors)
    right_unitary_four_tensors = unitary_order_four_right_tensors(mpo_tensors)
    
    left_unitary_two_tensors = list()
    left_unitary_four_tensors = list()
    left_overlap_tensors = list()

    num_sites = len(mpo_tensors)
    assert len(mpo_tensors) == len(b_tensors)

    # Leftmost site
    w = mpo_tensors[0]
    b = b_tensors[0]

    t = right_unitary_two_tensors[0]

    # Second order terms
    grad_2 = npc.tensordot(t, w, [['vL'], ['vR',]])

    order_2_score = update_mpo_score(
        grad_2,
        w,
        [['vL*',], ['vR*',]]
    )

    grad_2 = mpo_tensor_raw_to_gradient(grad_2, w)

    # Fourth order terms
    t = right_unitary_four_tensors[0]

    grad_4 = npc.tensordot(t, w, [['vL'], ['vR',]])
    grad_4 = npc.tensordot(grad_4, w.conj(), [['vL*', 'p'], ['vR*', 'p*']])
    grad_4 = npc.tensordot(grad_4, w, [['vL1', 'p'], ['vR', 'p*']])

    order_4_score = update_mpo_score(
        grad_4,
        w,
        [['vL1*',], ['vR*',]]
    )

    grad_4 = mpo_tensor_raw_to_gradient(grad_4, w)

    unitary_score = order_4_score - 2*order_2_score + total_dimension
    unitary_grad = (grad_4 - grad_2)/np.sqrt(1+unitary_score)
    
    # Overlap terms
    t = right_overlap_tensors[0].conj().replace_label('vLm*', 'vLm')

    grad_o = npc.tensordot(t, b, [['vL'], ['vR',]])
    grad_o = npc.tensordot(grad_o, b.conj(), [['vL*',], ['vR*',]])
    grad_o = npc.tensordot(grad_o, left_environment, [['vL', 'vL*'], ['vR', 'vR*']])

    c_conj = update_mpo_score(
        grad_o,
        w,
        [['vLm',], ['vR*',]],
        take_abs=True
    )
    c = c_conj.conjugate()
    c_abs = np.abs(c)
    
    grad_o_scale = c*(1 - overlap_target/c_abs)
    grad_o = grad_o_scale*grad_o
    grad_o = mpo_tensor_raw_to_gradient(grad_o, w)

    grad = (
        unitarity_learning_rate*unitary_grad +
        overlap_learning_rate*grad_o
    )
    adam_grad = adam_optimizers[0].update(grad)
    grads.append(adam_grad)

    # Create and save left tensors
    t = npc.tensordot(w, w.conj(), [['p', 'p*'], ['p*', 'p']])
    left_unitary_two_tensors.append(t)
    
    t = npc.tensordot(w, w.conj(), [['p',], ['p*',]])
    t.ireplace_labels(['vR', 'vR*'], ['vR1', 'vR1*'])
    t = npc.tensordot(t, w, [['p',], ['p*',]])
    t = npc.tensordot(t, w.conj(), [['p', 'p*'], ['p*', 'p']])
    
    left_unitary_four_tensors.append(t)

    t = npc.tensordot(b, w.conj(), [['p',], ['p*',]])
    #print(t)
    t.ireplace_label('vR*', 'vRm')
    #print(t)
    t = npc.tensordot(t, left_environment, [['vL',], ['vR',]])
    #print(t)
    t = npc.tensordot(t, b.conj(), [['vR*', 'p'], ['vL*', 'p*']])

    #print(t)

    left_overlap_tensors.append(t)

    # Inner sites
    for i in range(1, num_sites-1):
        w = mpo_tensors[i]
        b = b_tensors[i]
    
        right_two_tensor = right_unitary_two_tensors[i]
        right_four_tensor = right_unitary_four_tensors[i]
        right_overlap_tensor = right_overlap_tensors[i].conj().replace_label('vLm*', 'vLm')

        # Order two terms
        left_two_tensor = left_unitary_two_tensors[-1]

        grad_2 = npc.tensordot(right_two_tensor, w, [['vL'], ['vR',]])
        grad_2 = npc.tensordot(grad_2, left_two_tensor, [['vL'], ['vR',]])

        grad_2 = mpo_tensor_raw_to_gradient(grad_2, w)

        # Order four terms
        left_four_tensor = left_unitary_four_tensors[-1]

        grad_4 = npc.tensordot(right_four_tensor, w, [['vL'], ['vR',]])
        grad_4 = npc.tensordot(grad_4, w.conj(), [['vL*', 'p'], ['vR*', 'p*']])

        grad_4 = npc.tensordot(
            grad_4,
            w.replace_label('vL', 'vL1'),
            [['vL1', 'p'], ['vR', 'p*']]
        )

        grad_4 = npc.tensordot(
            grad_4,
            left_four_tensor,
            [['vL', 'vL*', 'vL1'], ['vR', 'vR*', 'vR1']]
        )

        grad_4 = mpo_tensor_raw_to_gradient(grad_4, w)

        unitary_grad = (grad_4 - grad_2)/np.sqrt(1+unitary_score)
    
        # Overlap terms
        left_overlap_tensor = left_overlap_tensors[-1]

        grad_o = npc.tensordot(right_overlap_tensor, b, [['vL',], ['vR',]])
        grad_o = npc.tensordot(grad_o, b.conj(), [['vL*',], ['vR*',]])
        grad_o = npc.tensordot(
            grad_o,
            left_overlap_tensor,
            [['vL*', 'vL'], ['vR', 'vR*',]]
        )

        grad_o = grad_o_scale*grad_o
        grad_o = mpo_tensor_raw_to_gradient(grad_o, w)

        grad = (
            unitarity_learning_rate*unitary_grad +
            overlap_learning_rate*grad_o
        )
        adam_grad = adam_optimizers[i].update(grad)
        grads.append(adam_grad)

        # Update left tensors
        t = npc.tensordot(left_two_tensor, w, [['vR',], ['vL']])
        t = npc.tensordot(
            t,
            w.conj(),
            [['vR*', 'p', 'p*'], ['vL*', 'p*', 'p']]
        )
        
        left_unitary_two_tensors.append(t)
        
        t = npc.tensordot(left_four_tensor, w, [['vR',], ['vL']])
        t = npc.tensordot(t, w.conj(), [['vR*', 'p'], ['vL*', 'p*']])
        t = npc.tensordot(
            t,
            w.replace_label('vR', 'vR1'),
            [['p', 'vR1'], ['p*', 'vL']]
        )
        t = npc.tensordot(
            t,
            w.conj().replace_label('vR*', 'vR1*'),
            [['p', 'p*', 'vR1*'], ['p*', 'p', 'vL*']]
        )
        
        left_unitary_four_tensors.append(t)

        t = left_overlap_tensor.ireplace_label('vR*', 'vR1*')
        t = npc.tensordot(
            left_overlap_tensor,
            w.conj(),
            [['vRm',], ['vL*']]
        )
        t.ireplace_label('vR*', 'vRm')
        t = npc.tensordot(t, b, [['vR', 'p*'], ['vL', 'p']])
        t = npc.tensordot(t, b.conj(), [['vR1*', 'p'], ['vL*', 'p*']])

        left_overlap_tensors.append(t)

    # Last site
    left_two_tensor = left_unitary_two_tensors[-1]
    w = mpo_tensors[-1]
    b = b_tensors[-1]
    
    grad_2 = npc.tensordot(left_two_tensor, w, [['vR'], ['vL',]])
    grad_2 = mpo_tensor_raw_to_gradient(grad_2, w)

    left_four_tensor = left_unitary_four_tensors[-1]
    
    grad_4 = npc.tensordot(left_four_tensor, w, [['vR'], ['vL',]])
    grad_4 = npc.tensordot(grad_4, w.conj(), [['vR*', 'p'], ['vL*', 'p*']])
    grad_4 = npc.tensordot(grad_4, w, [['vR1', 'p'], ['vL', 'p*']])

    grad_4 = mpo_tensor_raw_to_gradient(grad_4, w)
    
    unitary_grad = (grad_4 - grad_2)/np.sqrt(1+unitary_score)

    left_overlap_tensor = left_overlap_tensors[-1]
    right_overlap_tensor = right_overlap_tensors[-1].conj()

    grad_o = npc.tensordot(right_overlap_tensor, b, [['vL',], ['vR',]])
    grad_o = npc.tensordot(grad_o, b.conj(), [['vL*',], ['vR*',]])
    grad_o = npc.tensordot(
        grad_o,
        left_overlap_tensor,
        [['vL*', 'vL'], ['vR', 'vR*',]]
    )

    grad_o = grad_o_scale*grad_o
    grad_o = mpo_tensor_raw_to_gradient(grad_o, w)


    grad = (
        unitarity_learning_rate*unitary_grad +
        overlap_learning_rate*grad_o
    )
    adam_grad = adam_optimizers[-1].update(grad)
    grads.append(adam_grad)

    """
    for i, g in enumerate(grads):
        mpo_tensors[i] = mpo_tensors[i] - g
    """
    
    return (grads, unitary_score, c_abs)

In [46]:
def swap_left_right_indices(npc_array):
    left_right_pairs = {
        'vL': 'vR',
        'vR': 'vL',
        'vL*': 'vR*',
        'vR*': 'vL*'
    }

    leg_labels = npc_array.get_leg_labels()

    old_labels = [l for l in leg_labels if l in left_right_pairs]
    new_labels = [left_right_pairs[l] for l in old_labels]

    out = npc_array.replace_labels(old_labels, new_labels)

    return out

In [47]:
def two_sided_mpo_gradient_descent_sweep(left_mpo_tensors, right_mpo_tensors,
    left_b_tensors, right_b_tensors, left_total_dimension,
    right_total_dimension, unitarity_learning_rate, overlap_learning_rate,
    symmetry_transfer_matrix, left_adam_optimizers, right_adam_optimizers):

    # Compute left and right symmetry environments
    # Right symmetry environment for left side first
    right_overlap_tensors = overlap_right_tensors(
        right_mpo_tensors,
        right_b_tensors
    )
    right_symmetry_environment = npc.tensordot(
        symmetry_transfer_matrix,
        right_overlap_tensors[0],
        [['vR', 'vR*'], ['vL', 'vL*']]
    )
    right_symmetry_environment = swap_left_right_indices(right_symmetry_environment)

    # Left symmetry environment for right side
    left_overlap_tensors = overlap_right_tensors(
        left_mpo_tensors,
        left_b_tensors
    )
    left_symmetry_environment = npc.tensordot(
        symmetry_transfer_matrix,
        swap_left_right_indices(left_overlap_tensors[0]),
        [['vL', 'vL*'], ['vR', 'vR*']]
    )

    # Get right gradients
    right_grads, unitary_score, c_abs = mpo_gradient_descent_sweep(
        right_mpo_tensors,
        right_b_tensors,
        right_total_dimension,
        right_overlap_tensors[1:],
        unitarity_learning_rate,
        overlap_learning_rate,
        1,
        left_symmetry_environment,
        right_adam_optimizers
    )
    
    for i, g in enumerate(right_grads):
        right_mpo_tensors[i] = right_mpo_tensors[i] - g

    # Get left gradients
    left_grads, *_ = mpo_gradient_descent_sweep(
        left_mpo_tensors,
        left_b_tensors,
        left_total_dimension,
        left_overlap_tensors[1:],
        unitarity_learning_rate,
        overlap_learning_rate,
        1,
        right_symmetry_environment,
        left_adam_optimizers
    )

    for i, g in enumerate(left_grads):
        left_mpo_tensors[i] = left_mpo_tensors[i] - g

    return (unitary_score, c_abs)

In [48]:
def unitarity_error_from_subscores(order_two_score, order_four_score, dimension):
    return order_four_score - 2*order_two_score + dimension

In [49]:
def unitarity_errors_from_subscores(order_two_scores, order_four_scores,
                                    dimension):
    out = [
        unitarity_error_from_subscores(o2, o4, dimension)
        for o2, o4 in zip(order_two_scores, order_four_scores)
    ]
    
    return out

In [50]:
def initialize_optimization(num_sites, bond_dimension, symmetry_case,
    unitarity_learning_rate, overlap_learning_rate, adam_params):
    right_b_tensors = [
        symmetry_case.psi.get_B(i)
        for i in range(
            symmetry_case.right_symmetry_index + 1,
            symmetry_case.right_symmetry_index + 1 + num_sites
        )
    ]

    left_b_tensors = [
        symmetry_case.psi.get_B(i, form='A')
        for i in range(
            symmetry_case.left_symmetry_index - 1,
            symmetry_case.left_symmetry_index - 1 - num_sites, -1
        )
    ]

    left_b_tensors = [
        swap_left_right_indices(b) for b in left_b_tensors
    ]

    right_physical_dims = [
        get_physical_dim(b) for b in right_b_tensors
    ]
    
    right_total_dimension = reduce(mul, right_physical_dims)

    left_physical_dims = [
        get_physical_dim(b) for b in left_b_tensors
    ]

    left_total_dimension = reduce(mul, left_physical_dims)

    virtual_dims = (
        [(None, bond_dimension),] +
        [(bond_dimension, bond_dimension)]*(num_sites - 2) +
        [(bond_dimension, None),]
    )

    right_mpo_tensors = get_random_mpo_tensors(
        right_physical_dims,
        virtual_dims
    )
    left_mpo_tensors = get_random_mpo_tensors(
        left_physical_dims,
        virtual_dims
    )
    rescale_mpo_tensors(right_mpo_tensors, 1)
    rescale_mpo_tensors(left_mpo_tensors, 1)

    symmetry_transfer_matrix = symmetry_case.npc_symmetry_transfer_matrix

    left_adam_optimizers = [
        AdamTenpy(*adam_params) for _ in range(num_sites)
    ]

    right_adam_optimizers = [
        AdamTenpy(*adam_params) for _ in range(num_sites)
    ]

    return (
        left_mpo_tensors,
        right_mpo_tensors,
        left_b_tensors,
        right_b_tensors,
        left_total_dimension,
        right_total_dimension,
        unitarity_learning_rate,
        overlap_learning_rate,
        symmetry_transfer_matrix,
        left_adam_optimizers,
        right_adam_optimizers
    )

### SPT phase extraction

In [51]:
def conjugate_single_mpo_tensor(mpo_tensor):
    leg_labels = mpo_tensor.get_leg_labels()

    if not ('vR' in leg_labels):
        return mpo_tensor.conj().replace_labels(['vL*',], ['vL',])
    elif not ('vL' in leg_labels):
        return mpo_tensor.conj().replace_labels(['vR*'], ['vR'])
    else:
        return mpo_tensor.conj().replace_labels(['vL*', 'vR*'], ['vL', 'vR'])

In [52]:
def conjugate_mpo(mpo_tensors):
    return [
        conjugate_single_mpo_tensor(t) for t in mpo_tensors
    ]

In [121]:
def mpo_product_expectation(left_environment, b_tensors, mpos):
    """
    Have an optional b_conj argument? Left and right arguments?

    Like a previous function we wrote?
    """

    # First site
    t = b_tensors[0]

    for i, l in enumerate(mpos):
        w = l[0]
        #print(t)
        #print(w)
        t = npc.tensordot(
            t,
            w.replace_label('vR', f'vR{i}'),
            [['p',], ['p*',]]
        )

    t = npc.tensordot(
        t,
        b_tensors[0].conj(),
        [['p',], ['p*',]]
    )

    t = npc.tensordot(
        t,
        left_environment,
        [['vL', 'vL*',], ['vR', 'vR*']]
    )

    # Inner sites
    for i in range(1, len(b_tensors)-1):
        t = npc.tensordot(
            t,
            b_tensors[i],
            [['vR',], ['vL',]]
        )

        for j, l in enumerate(mpos):
            w = l[i]
            t = npc.tensordot(
                t,
                w.replace_label('vR', f'vR{j}'),
                [['p', f'vR{j}'], ['p*', 'vL']]
            )
    
        t = npc.tensordot(
            t,
            b_tensors[i].conj(),
            [['p', 'vR*'], ['p*', 'vL*']]
        )

    # Last site
    t = npc.tensordot(
        t,
        b_tensors[-1],
        [['vR',], ['vL',]]
    )

    for i, l in enumerate(mpos):
        w = l[-1]
        t = npc.tensordot(
            t,
            w,
            [['p', f'vR{i}'], ['p*', 'vL']]
        )

    t = npc.tensordot(
        t,
        b_tensors[-1].conj(),
        [['p', 'vR*', 'vR'], ['p*', 'vL*', 'vR*']]
    )

    return t

In [54]:
def proj_rep_phase(left_environment, b_tensors, mpo_1, mpo_2, mpo_3):
    num = mpo_product_expectation(
        left_environment,
        b_tensors,
        [
            mpo_1,
            mpo_2,
            conjugate_mpo(mpo_3)
        ]
    )

    den = npc.trace(left_environment)

    return num/den

# Load MPO solutions
## Triv proj rep solutions
### Random search solutions

In [56]:
MPO_SOLUTIONS_DIR = r'solutions/triv_hyperparameter_search'

In [60]:
HyperParams = namedtuple(
    'HyperParams',
    ['alpha', 'beta_1', 'beta_2', 'overlap_learning_rate']
)

In [65]:
def parse_random_search_dict(d):
    for k, v in d.items():
        if v['good_sol']:
            return (k, v)
    return None

In [72]:
random_sols = dict()
empty_cases = list()

for file_name in os.listdir(MPO_SOLUTIONS_DIR):
    with open(rf'{MPO_SOLUTIONS_DIR}/{file_name}', 'rb') as file:
        d = pickle.load(file)
    out = parse_random_search_dict(d)

    if out is None:
        empty_cases.append(file_name)
    else:
        k, v = out

        a, b, c = [
            int(s)
            for s in (file_name.split('.')[0]).split('_')
        ]
        a/=100
        random_sols[(a,b,c)] = {**(k._asdict()), **v}

In [73]:
empty_cases

[]

All solutions found!

In [80]:
final_scores_dict = dict()

for k, v in random_sols.items():
    overlap = np.round(v['overlap_scores'][-1], 3)
    unitarity = np.round(np.real(v['unitarity_scores'][-1]), 3)

    final_scores_dict[k] = (unitarity, overlap)

In [81]:
final_scores_dict

{(0.15, 1, 2): (4.858, 0.943),
 (0.15, 1, 3): (4.967, 0.919),
 (0.15, 1, 1): (4.994, 0.932),
 (0.2, 0, 3): (4.866, 0.956),
 (0.05, 0, 1): (2.495, 0.901),
 (0.4, 1, 3): (4.869, 0.956),
 (0.5, 1, 3): (2.838, 0.9),
 (0.5, 1, 1): (4.848, 0.933),
 (0.2, 1, 3): (4.987, 0.901),
 (0.05, 1, 2): (4.922, 0.903),
 (0.2, 1, 1): (4.984, 0.945),
 (0.5, 0, 1): (4.922, 0.965),
 (0.5, 0, 3): (4.976, 0.954),
 (0.25, 1, 1): (4.94, 0.954),
 (0.0, 1, 2): (4.987, 0.934),
 (0.1, 0, 2): (4.664, 0.949),
 (0.35, 1, 2): (4.906, 0.949),
 (0.0, 1, 1): (4.779, 0.957),
 (0.35, 1, 3): (4.91, 0.923),
 (0.45, 1, 3): (4.997, 0.964),
 (0.35, 0, 1): (4.998, 0.949)}

These all look good. Pull in other solutions.

### Fixed hyperparameter solutions

In [82]:
with open(r'solutions/non_trivial_models_triv_proj_rep_mpo_sol.pkl', 'rb') as file:
    dict_triv = pickle.load(file)

In [83]:
dict_triv.keys()

dict_keys(['solutions', 'unitarity_scores', 'overlaps'])

In [86]:
random_sols[(0, 1, 1)].keys()

dict_keys(['alpha', 'beta_1', 'beta_2', 'overlap_learning_rate', 'solutions', 'unitarity_scores', 'overlap_scores', 'time', 'good_sol'])

In [88]:
fixed_sols = dict()

for k, sols in dict_triv['solutions'].items():
    if k not in random_sols:
        fixed_sols[k] = {
            'solutions': sols,
            'unitarity_scores': dict_triv['unitarity_scores'][k],
            'overlap_scores': dict_triv['overlaps'][k]
        }

In [89]:
final_scores_dict = dict()

for k, v in fixed_sols.items():
    overlap = np.round(v['overlap_scores'][-1], 3)
    unitarity = np.round(np.real(v['unitarity_scores'][-1]), 3)

    final_scores_dict[k] = (unitarity, overlap)

In [91]:
final_scores_dict

{(0.65, 0, 1): (0.156, 1.009),
 (0.65, 0, 2): (0.113, 0.992),
 (0.65, 0, 3): (0.056, 0.986),
 (0.65, 1, 1): (0.131, 1.003),
 (0.65, 1, 2): (0.109, 1.005),
 (0.65, 1, 3): (0.146, 0.983),
 (0.45, 0, 1): (0.593, 0.949),
 (0.45, 0, 2): (0.761, 0.958),
 (0.45, 0, 3): (0.913, 0.914),
 (0.45, 1, 1): (2.38, 0.906),
 (0.45, 1, 2): (0.181, 0.971),
 (0.55, 0, 1): (0.557, 0.99),
 (0.55, 0, 2): (0.135, 0.984),
 (0.55, 0, 3): (0.797, 0.99),
 (0.55, 1, 1): (0.59, 0.975),
 (0.55, 1, 2): (0.125, 0.981),
 (0.55, 1, 3): (0.608, 0.984),
 (0.95, 0, 1): (0.028, 1.002),
 (0.95, 0, 2): (0.127, 1.009),
 (0.95, 0, 3): (0.066, 0.985),
 (0.95, 1, 1): (0.031, 1.003),
 (0.95, 1, 2): (0.113, 0.986),
 (0.95, 1, 3): (0.046, 1.002),
 (0.3, 0, 1): (0.145, 0.992),
 (0.3, 0, 2): (0.153, 0.986),
 (0.3, 0, 3): (0.071, 0.98),
 (0.3, 1, 1): (0.191, 0.976),
 (0.3, 1, 2): (0.101, 0.991),
 (0.3, 1, 3): (0.158, 0.956),
 (0.7, 0, 1): (0.1, 0.986),
 (0.7, 0, 2): (0.105, 1.006),
 (0.7, 0, 3): (0.173, 1.009),
 (0.7, 1, 1): (0.093, 0.

Looks good!

In [92]:
(len(fixed_sols), len(random_sols))

(105, 21)

In [93]:
triv_sols_dict = {**fixed_sols, **random_sols}

In [94]:
len(triv_sols_dict)

126

## Non trivial proj rep solutions

In [98]:
MPO_SOLUTIONS_DIR = r'solutions/non_triv_hyperparameter_search'

In [99]:
random_sols = dict()
empty_cases = list()

for file_name in os.listdir(MPO_SOLUTIONS_DIR):
    with open(rf'{MPO_SOLUTIONS_DIR}/{file_name}', 'rb') as file:
        d = pickle.load(file)
    out = parse_random_search_dict(d)

    if out is None:
        empty_cases.append(file_name)
    else:
        k, v = out

        a, b, c = [
            int(s)
            for s in (file_name.split('.')[0]).split('_')
        ]
        a/=100
        random_sols[(a,b,c)] = {**(k._asdict()), **v}

In [100]:
empty_cases

[]

In [101]:
final_scores_dict = dict()

for k, v in random_sols.items():
    overlap = np.round(v['overlap_scores'][-1], 3)
    unitarity = np.round(np.real(v['unitarity_scores'][-1]), 3)

    final_scores_dict[k] = (unitarity, overlap)

In [102]:
final_scores_dict

{(1.0, 0, 3): (4.955, 0.912),
 (0.15, 1, 2): (4.891, 0.951),
 (0.9, 0, 3): (4.866, 0.916),
 (0.05, 0, 2): (4.976, 0.947),
 (0.8, 0, 3): (4.95, 0.97),
 (0.8, 0, 2): (4.947, 0.956),
 (0.05, 0, 3): (4.969, 0.957),
 (0.9, 0, 2): (4.933, 0.972),
 (0.3, 0, 1): (0.992, 0.935),
 (0.2, 0, 1): (4.992, 0.972),
 (1.0, 0, 2): (4.986, 0.908),
 (0.15, 1, 3): (4.828, 0.939),
 (0.15, 1, 1): (4.935, 0.958),
 (0.3, 0, 3): (4.866, 0.957),
 (0.2, 0, 3): (4.859, 0.941),
 (0.05, 0, 1): (4.895, 0.923),
 (0.9, 0, 1): (4.988, 0.914),
 (0.8, 0, 1): (4.981, 0.942),
 (0.2, 0, 2): (4.955, 0.966),
 (0.3, 0, 2): (0.989, 0.966),
 (1.0, 0, 1): (4.979, 0.959),
 (0.65, 0, 1): (4.638, 0.908),
 (0.75, 0, 1): (4.938, 0.964),
 (0.4, 1, 3): (4.933, 0.922),
 (0.5, 1, 3): (4.965, 0.976),
 (0.5, 1, 2): (4.914, 0.929),
 (0.4, 1, 2): (4.937, 0.964),
 (0.75, 0, 2): (4.801, 0.926),
 (0.65, 0, 2): (4.98, 0.956),
 (0.4, 1, 1): (4.967, 0.938),
 (0.5, 1, 1): (4.901, 0.938),
 (0.65, 0, 3): (4.952, 0.951),
 (0.75, 0, 3): (4.985, 0.909),
 

# Calculate fermionic cohomology

In [171]:
def right_mpo_fp_charge(left_environment, b_tensors, mpo_tensors):
    fp_bs = list()
    
    for i, b in enumerate(b_tensors):
        dim = get_physical_dim(b)
        if dim%2:
            b_to_add = npc.tensordot(b, npc_JW, [['p',], ['p*',]])
        else:
            b_to_add = b
    
        fp_bs.append(b_to_add)

    fp_exp = mpo_product_expectation(
        left_environment,
        fp_bs,
        [mpo_tensors,]
    )
    exp = mpo_product_expectation(
        left_environment,
        b_tensors,
        [mpo_tensors,]
    )

    relative_phase = fp_exp/exp

    return (relative_phase, exp)

In [213]:
test_key = (0.3, 0, 2)

In [214]:
test_case = cases_triv_proj_rep[test_key]

In [215]:
test_left_mpo, test_right_mpo = triv_sols_dict[test_key]['solutions']

In [216]:
test_left_mpo

[<npc.Array shape=(2, 2, 5) labels=['p', 'p*', 'vR']>,
 <npc.Array shape=(4, 4, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(2, 2, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(4, 4, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(2, 2, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(4, 4, 5) labels=['p', 'p*', 'vL']>]

In [217]:
test_right_mpo

[<npc.Array shape=(4, 4, 5) labels=['p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(4, 4, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(2, 2, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(4, 4, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(2, 2, 5) labels=['p', 'p*', 'vL']>]

In [218]:
num_sites = 6

In [219]:
test_b_tensors = [
    test_case.psi.get_B(i)
    for i in range(
        test_case.right_symmetry_index + 1,
        test_case.right_symmetry_index + 1 + len(test_right_mpo)
    )
]

In [220]:
right_mpo_fp_charge(
    test_case.right_projected_symmetry_state,
    test_b_tensors,
    test_right_mpo
)

((1-0j), (-1.4164452368361067-0.236791102495156j))

Just need to compute the left/right environments now, and loop.

In [180]:
len(triv_sols_dict)

126

In [181]:
type(test_case)

SPTOptimization.SymmetryActionWithBoundaryUnitaries.SymmetryActionWithBoundaryUnitaries

In [182]:
triv_proj_rep_fp_charges = dict()

for k, d in triv_sols_dict.items():
    left_mpo, right_mpo = d['solutions']
    case = cases_triv_proj_rep[k]

    right_b_tensors = [
        case.psi.get_B(i)
        for i in range(
            case.right_symmetry_index + 1,
            case.right_symmetry_index + 1 + len(right_mpo)
        )
    ]

    left_b_tensors = [
        case.psi.get_B(i, form='A')
        for i in range(
            case.left_symmetry_index - 1,
            case.left_symmetry_index - 1 - num_sites, -1
        )
    ]

    left_b_tensors = [
        swap_left_right_indices(b)
        for b in left_b_tensors
    ]

    # Compute left and right symmetry environments
    # Right symmetry environment for left side first
    right_overlap_tensors = overlap_right_tensors(
        right_mpo,
        right_b_tensors
    )
    right_symmetry_environment = npc.tensordot(
        case.npc_symmetry_transfer_matrix,
        right_overlap_tensors[0],
        [['vR', 'vR*'], ['vL', 'vL*']]
    )
    right_symmetry_environment = swap_left_right_indices(right_symmetry_environment)

    # Left symmetry environment for right side
    left_overlap_tensors = overlap_right_tensors(
        left_mpo,
        left_b_tensors
    )
    left_symmetry_environment = npc.tensordot(
        case.npc_symmetry_transfer_matrix,
        swap_left_right_indices(left_overlap_tensors[0]),
        [['vL', 'vL*'], ['vR', 'vR*']]
    )

    right_fp_charge = right_mpo_fp_charge(
        left_symmetry_environment,
        right_b_tensors,
        right_mpo
    )

    left_fp_charge = right_mpo_fp_charge(
        right_symmetry_environment,
        left_b_tensors,
        left_mpo
    )

    triv_proj_rep_fp_charges[k] = (*left_fp_charge, *right_fp_charge)

In [183]:
data = [(*k, *v) for k, v in triv_proj_rep_fp_charges.items()]

triv_proj_rep_fp_charges_1_df = pd.DataFrame(
    data,
    columns=['t', 'jw', 'a', 'fp_l', 'ol', 'fp_r', 'or']
)

In [184]:
triv_proj_rep_fp_charges_1_df['fp_l'].value_counts()

fp_l
1.0-0.0j    104
1.0+0.0j     14
1.0-0.0j      1
1.0-0.0j      1
1.0+0.0j      1
1.0-0.0j      1
1.0-0.0j      1
1.0-0.0j      1
1.0+0.0j      1
1.0-0.0j      1
Name: count, dtype: int64

In [185]:
triv_proj_rep_fp_charges_1_df['fp_r'].value_counts()

fp_r
1.0-0.0j    105
1.0+0.0j     13
1.0+0.0j      1
1.0-0.0j      1
1.0+0.0j      1
1.0-0.0j      1
1.0+0.0j      1
1.0+0.0j      1
1.0+0.0j      1
1.0+0.0j      1
Name: count, dtype: int64

In [222]:
triv_proj_rep_fp_charges = dict()

for k, d in triv_sols_dict.items():
    left_mpo, right_mpo = d['solutions']
    case = cases_triv_proj_rep[k]

    right_b_tensors = [
        case.psi.get_B(i)
        for i in range(
            case.right_symmetry_index + 1,
            case.right_symmetry_index + 1 + len(right_mpo)
        )
    ]

    left_b_tensors = [
        case.psi.get_B(i, form='A')
        for i in range(
            case.left_symmetry_index - 1,
            case.left_symmetry_index - 1 - num_sites, -1
        )
    ]

    left_b_tensors = [
        swap_left_right_indices(b)
        for b in left_b_tensors
    ]

    right_symmetry_environment = case.left_projected_symmetry_state
    right_symmetry_environment = swap_left_right_indices(right_symmetry_environment)

    left_symmetry_environment = case.right_projected_symmetry_state

    right_fp_charge = right_mpo_fp_charge(
        left_symmetry_environment,
        right_b_tensors,
        right_mpo
    )

    left_fp_charge = right_mpo_fp_charge(
        right_symmetry_environment,
        left_b_tensors,
        left_mpo
    )

    triv_proj_rep_fp_charges[k] = (
        *left_fp_charge,
        *right_fp_charge,
        case.symmetry_transfer_matrix_singular_vals[0]
    )

In [223]:
data = [(*k, *v) for k, v in triv_proj_rep_fp_charges.items()]

triv_proj_rep_fp_charges_2_df = pd.DataFrame(
    data,
    columns=['t', 'jw', 'a', 'fp_l', 'ol', 'fp_r', 'or', 'sing_val']
)

In [224]:
triv_proj_rep_fp_charges_2_df['fp_l'].value_counts()

fp_l
1.0-0.0j    109
1.0-0.0j      9
1.0+0.0j      1
1.0-0.0j      1
1.0-0.0j      1
1.0-0.0j      1
1.0+0.0j      1
1.0+0.0j      1
1.0+0.0j      1
1.0-0.0j      1
Name: count, dtype: int64

In [225]:
triv_proj_rep_fp_charges_2_df['fp_r'].value_counts()

fp_r
1.0+0.0j    106
1.0-0.0j     12
1.0+0.0j      1
1.0-0.0j      1
1.0-0.0j      1
1.0+0.0j      1
1.0+0.0j      1
1.0-0.0j      1
1.0-0.0j      1
1.0+0.0j      1
Name: count, dtype: int64

In [226]:
triv_proj_rep_fp_charges_2_df['overlap'] = (
    np.abs(triv_proj_rep_fp_charges_2_df['ol'])
    *np.abs(triv_proj_rep_fp_charges_2_df['or'])
    *np.abs(triv_proj_rep_fp_charges_2_df['sing_val'])
)

In [227]:
triv_proj_rep_fp_charges_2_df

,t,jw,a,fp_l,ol,fp_r,or,sing_val,overlap
0,0.65,0,1,1.0+0.0j,-0.884291-0.455162j,1.0+0.0j,1.044370+0.048644j,0.959778,0.997987
1,0.65,0,2,1.0-0.0j,-0.954166-0.355204j,1.0+0.0j,0.807514-0.615764j,0.959778,0.992333
2,0.65,0,3,1.0+0.0j,0.523254+0.920771j,1.0+0.0j,0.578385+0.788813j,0.959778,0.994244
3,0.65,1,1,1.0+0.0j,0.896500-0.343312j,1.0-0.0j,-1.001324-0.397012j,0.959778,0.992465
4,0.65,1,2,1.0+0.0j,1.022346+0.244310j,1.0+0.0j,-0.683846+0.703706j,0.959778,0.989935
...,...,...,...,...,...,...,...,...,...
121,0.35,1,2,1.0-0.0j,-0.012559-0.015797j,1.0-0.0j,-0.008040-0.006602j,0.495951,0.000104
122,0.00,1,1,1.0-0.0j,-0.661205-1.578544j,1.0-0.0j,0.618881-1.458487j,0.500000,1.355761
123,0.35,1,3,1.0+0.0j,-0.912589+1.276062j,1.0-0.0j,-1.573281-0.015692j,0.495951,1.224153
124,0.45,1,3,1.0-0.0j,-1.163135-0.764176j,1.0+0.0j,-0.280965+1.236691j,0.467965,0.825944


In [229]:
triv_proj_rep_fp_charges_2_df.sort_values('overlap').to_csv('mpo_overlaps.csv')

In [191]:
triv_proj_rep_fp_charges_1_df

,t,jw,a,fp_l,ol,fp_r,or
0,0.65,0,1,1.0-0.0j,-0.865130-0.497522j,1.0+0.0j,-0.865130-0.497522j
1,0.65,0,2,1.0-0.0j,-0.949435+0.288614j,1.0-0.0j,-0.949435+0.288614j
2,0.65,0,3,1.0+0.0j,-0.406633+0.907288j,1.0+0.0j,-0.406633+0.907288j
3,0.65,1,1,1.0-0.0j,-0.992397-0.011666j,1.0-0.0j,-0.992397-0.011666j
4,0.65,1,2,1.0-0.0j,-0.836014+0.530144j,1.0-0.0j,-0.836014+0.530144j
...,...,...,...,...,...,...,...
121,0.35,1,2,1.0-0.0j,-0.000002+0.000104j,1.0+0.0j,-0.000002+0.000104j
122,0.00,1,1,1.0-0.0j,-1.355746-0.006286j,1.0+0.0j,-1.355746-0.006286j
123,0.35,1,3,1.0-0.0j,0.721996-0.988571j,1.0-0.0j,0.721996-0.988571j
124,0.45,1,3,1.0+0.0j,0.595181-0.572664j,1.0+0.0j,0.595181-0.572664j


In [193]:
triv_proj_rep_fp_charges_1_df['abs_ol'] = np.abs(triv_proj_rep_fp_charges_1_df['ol'])
triv_proj_rep_fp_charges_1_df['abs_or'] = np.abs(triv_proj_rep_fp_charges_1_df['or'])
triv_proj_rep_fp_charges_2_df['abs_ol'] = np.abs(triv_proj_rep_fp_charges_2_df['ol'])
triv_proj_rep_fp_charges_2_df['abs_or'] = np.abs(triv_proj_rep_fp_charges_2_df['or'])

In [194]:
triv_proj_rep_fp_charges_1_df

,t,jw,a,fp_l,ol,fp_r,or,abs_ol,abs_or
0,0.65,0,1,1.0-0.0j,-0.865130-0.497522j,1.0+0.0j,-0.865130-0.497522j,0.997987,0.997987
1,0.65,0,2,1.0-0.0j,-0.949435+0.288614j,1.0-0.0j,-0.949435+0.288614j,0.992333,0.992333
2,0.65,0,3,1.0+0.0j,-0.406633+0.907288j,1.0+0.0j,-0.406633+0.907288j,0.994244,0.994244
3,0.65,1,1,1.0-0.0j,-0.992397-0.011666j,1.0-0.0j,-0.992397-0.011666j,0.992465,0.992465
4,0.65,1,2,1.0-0.0j,-0.836014+0.530144j,1.0-0.0j,-0.836014+0.530144j,0.989935,0.989935
...,...,...,...,...,...,...,...,...,...
121,0.35,1,2,1.0-0.0j,-0.000002+0.000104j,1.0+0.0j,-0.000002+0.000104j,0.000104,0.000104
122,0.00,1,1,1.0-0.0j,-1.355746-0.006286j,1.0+0.0j,-1.355746-0.006286j,1.355761,1.355761
123,0.35,1,3,1.0-0.0j,0.721996-0.988571j,1.0-0.0j,0.721996-0.988571j,1.224153,1.224153
124,0.45,1,3,1.0+0.0j,0.595181-0.572664j,1.0+0.0j,0.595181-0.572664j,0.825944,0.825944


In [231]:
np.max(np.abs(
    np.abs(triv_proj_rep_fp_charges_1_df['ol'])
    - np.abs(triv_proj_rep_fp_charges_1_df['or'])
))

1.3322676295501878e-15

In [195]:
triv_proj_rep_fp_charges_2_df

,t,jw,a,fp_l,ol,fp_r,or,abs_ol,abs_or
0,0.65,0,1,1.0+0.0j,-0.884291-0.455162j,1.0+0.0j,1.044370+0.048644j,0.994556,1.045502
1,0.65,0,2,1.0-0.0j,-0.954166-0.355204j,1.0+0.0j,0.807514-0.615764j,1.018137,1.015502
2,0.65,0,3,1.0+0.0j,0.523254+0.920771j,1.0+0.0j,0.578385+0.788813j,1.059063,0.978139
3,0.65,1,1,1.0+0.0j,0.896500-0.343312j,1.0-0.0j,-1.001324-0.397012j,0.959987,1.077158
4,0.65,1,2,1.0+0.0j,1.022346+0.244310j,1.0+0.0j,-0.683846+0.703706j,1.051132,0.981248
...,...,...,...,...,...,...,...,...,...
121,0.35,1,2,1.0-0.0j,-0.012559-0.015797j,1.0-0.0j,-0.008040-0.006602j,0.020181,0.010403
122,0.00,1,1,1.0-0.0j,-0.661205-1.578544j,1.0-0.0j,0.618881-1.458487j,1.711430,1.584360
123,0.35,1,3,1.0+0.0j,-0.912589+1.276062j,1.0-0.0j,-1.573281-0.015692j,1.568806,1.573359
124,0.45,1,3,1.0-0.0j,-1.163135-0.764176j,1.0+0.0j,-0.280965+1.236691j,1.391707,1.268206


# Debug - recreate overlaps

In [241]:
test_key

(0.3, 0, 2)

In [242]:
file_name = '30_0_2.pkl'

In [266]:
file_name

'30_0_2.pkl'

In [283]:
triv_sols_dict[test_key]['overlap_scores'][-1]

0.9861950174347379

In [285]:
left_mpo_tensors, right_mpo_tensors = triv_sols_dict[test_key]['solutions']

In [286]:
left_mpo_tensors

[<npc.Array shape=(2, 2, 5) labels=['p', 'p*', 'vR']>,
 <npc.Array shape=(4, 4, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(2, 2, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(4, 4, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(2, 2, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(4, 4, 5) labels=['p', 'p*', 'vL']>]

In [287]:
right_mpo_tensors

[<npc.Array shape=(4, 4, 5) labels=['p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(4, 4, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(2, 2, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(4, 4, 5, 5) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(2, 2, 5) labels=['p', 'p*', 'vL']>]

In [75]:
def two_sided_mpo_expectation(symmetry_case, left_mpo_tensors,
    right_mpo_tensors):
    case = symmetry_case

    left_b_tensors = [
        case.psi.get_B(i, form='A')
        for i in range(
            case.left_symmetry_index - 1,
            case.left_symmetry_index - 1 - len(left_mpo_tensors), -1
        )
    ]

    left_b_tensors = [
        swap_left_right_indices(b)
        for b in left_b_tensors
    ]

    right_b_tensors = [
        case.psi.get_B(i)
        for i in range(
            case.right_symmetry_index + 1,
            case.right_symmetry_index + 1 + len(right_mpo_tensors)
        )
    ]

    t = right_b_tensors[-1]

    t = npc.tensordot(
        t,
        right_mpo_tensors[-1].replace_label('vL', 'vLm'),
        [['p',], ['p*']]
    )

    t = npc.tensordot(
        t,
        right_b_tensors[-1].conj(),
        [['p', 'vR'], ['p*', 'vR*']]
    )

    for b, w in zip(right_b_tensors[-2:0:-1], right_mpo_tensors[-2:0:-1]):
        t = npc.tensordot(t, b, [['vL',], ['vR']])
        t = npc.tensordot(
            t,
            w.replace_label('vL', 'vLm'),
            [['p', 'vLm'], ['p*', 'vR']]
        )
        t = npc.tensordot(t, b.conj(), [['p', 'vL*',], ['p*', 'vR*']])

    t = npc.tensordot(t, right_b_tensors[0], [['vL',], ['vR']])
    t = npc.tensordot(
        t,
        right_mpo_tensors[0],
        [['p', 'vLm'], ['p*', 'vR']]
    )
    t = npc.tensordot(t, right_b_tensors[0].conj(), [['p', 'vL*',], ['p*', 'vR*']])
    
    
    t = npc.tensordot(
        t,
        symmetry_case.npc_symmetry_transfer_matrix,
        [['vL', 'vL*'], ['vR', 'vR*']]
    )

    t = swap_left_right_indices(t)

    t = npc.tensordot(
        t,
        left_b_tensors[0],
        [['vR',], ['vL',]]
    )

    t = npc.tensordot(
        t,
        left_mpo_tensors[0].replace_label('vR', 'vRm'),
        [['p',], ['p*',]]
    )

    t = npc.tensordot(
        t,
        left_b_tensors[0].conj(),
        [['p', 'vR*'], ['p*', 'vL*']]
    )

    for b, w in zip(left_b_tensors[1:-1], left_mpo_tensors[1:-1]):
        t = npc.tensordot(t, b, [['vR',], ['vL']])
        t = npc.tensordot(
            t,
            w.replace_label('vR', 'vRm'),
            [['p', 'vRm'], ['p*', 'vL']]
        )
        t = npc.tensordot(t, b.conj(), [['p', 'vR*',], ['p*', 'vL*']])

    t = npc.tensordot(t, left_b_tensors[-1], [['vR',], ['vL']])
    t = npc.tensordot(
        t,
        left_mpo_tensors[-1],
        [['p', 'vRm'], ['p*', 'vL']]
    )
    t = npc.tensordot(t, left_b_tensors[-1].conj(), [['p', 'vR*',], ['p*', 'vL*']])

    out = npc.trace(t)

    return out

In [291]:
out = two_sided_mpo_expectation(
    cases_triv_proj_rep[test_key],
    left_mpo_tensors,
    right_mpo_tensors
)

In [292]:
np.abs(out)

0.9784873091677865

In [294]:
test_triv_overlaps_dict = dict()

for k, v in triv_sols_dict.items():
    left_mpo_tensors, right_mpo_tensors = v['solutions']

    overlap = two_sided_mpo_expectation(
        cases_triv_proj_rep[k],
        left_mpo_tensors,
        right_mpo_tensors
    )

    test_triv_overlaps_dict[k] = (
        v['overlap_scores'][-1],
        overlap
    )

In [296]:
X = np.array(list(test_triv_overlaps_dict.values()))

In [299]:
aX = np.abs(X)
(
    np.max(np.abs(aX[:, 1] - aX[:, 0])),
    np.argmax(np.abs(aX[:, 1] - aX[:, 0]))
)

(0.9646434119411333, 116)

In [300]:
X[116]

array([9.64643414e-01+0.00000000e+00j, 1.89841814e-09+4.56017427e-10j])

In [302]:
list(test_triv_overlaps_dict.keys())[116]

(0.5, 0, 1)

In [304]:
pd.Series(np.abs(aX[:, 1] - aX[:, 0])).describe()

count    126.000000
mean       0.120148
std        0.275545
min        0.000204
25%        0.007387
50%        0.016741
75%        0.022482
max        0.964643
dtype: float64